# Realizzazione di un modello 3D a partire dalla sua pianta in 2D

### Modello 2D scelto:

<img src="files/myPlanReverse.jpeg" width="400">
<img src="files/texturedPlan1.jpg" width="600">

#### Link : https://www.drawpro.com/wp-content/uploads/2013/10/Architectural-2-800x1100.png
(n.b. per ragioni pratiche è stata utilizzata l'immagine speculare al modello originale)

## Modello 3D riprodotto:

### Modello 3D semplice (mura + vani porta/finestra):
<img src="files/simplePlan1.jpg" width="800">
<img src="files/simplePlan2.jpg" width="800">
<img src="files/simplePlan3.jpg" width="800">

### Modello 3D con aggiunta di porte e finestre:
<img src="files/intermediatePlan1.jpg" width="800">
<img src="files/intermediatePlan2.jpg" width="800">

### Modello 3D con aggiunta di verande e basamento:
<img src="files/finalPlan1.jpg" width="800">
<img src="files/finalPlan2.jpg" width="800" >
<img src="files/finalPlan3.jpg" width="800">

### Modello 3D con aggiunta di texture:
<img src="files/texturedPlan1.jpg" >
<img src="files/texturedPlan2.jpg" >


Importo la libreria pyplasm e definisco un coefficiente di scala

In [154]:
from pyplasm import *
scale = 50

Definisco una funzione che mi permette di realizzare una pianta 2D del modello scelto a partire da un file .lines (che contiene per ogni sua riga 2 punti espressi mediante 2 coordinate ciascuno: ogni coppia di punti rappresenta sul piano XY un segmento della pianta 2D).
Utilizzo questa funzione per ottenere sia le mura del mio modello sia lo spazio occupato dalle porte/finestre da "sottrarre" alle suddette mura.

In [155]:
def lines2structure(fileName):
    """
    Funzione che legge coppie di punti da file e realizza una pianta 2D a partire dal modello importato
    
    Args:
        fileName(stringa): è il path+nome_del_file da cui la funzione estrapola le coordinate dei punti per tracciare i segmenti
        
    Returns:
        wall(oggetto hpc): è una rappresentazione "filiforme" sul piano XY del modello importato
    """
    import csv
    inizialized = False
    with open (fileName, 'rb') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in spamreader:
                points = []
                for val in row:
                    points.append(float(val)/scale)
                #print points
                if not inizialized: 
                    wall = MKPOL([[[points[0], points[1],0], [points[2], points[3],0]],[[1,2]],[[1]]])
                    inizialized = True
                else :  wall = STRUCT([wall, MKPOL([[[points[0], points[1],0], [points[2], points[3],0]],[[1,2]],[[1]]])])
    return wall  

Definisco una funzione che analogmente a quella precedente mi permette di localizzare a partire da un file .lines lo spazio 2D occupato da una porta messa in "diagonale" (ovvero nè perfettamente parallela all'asse X nè all'asse Y)

In [156]:
def diagonalDoor(fileName):
    """
    Funzione che identifica lo spazio occupato da una porta diagonale: a partire da un file traccia la proiezione 2D sul piano XY 
    di tale porta
    
    Args:
        fileName(stringa): è il path+nome_del_file da cui la funzione estrapola i segmenti che compongono la proiezione
        
    Returns:
        diagonalDoorSpace(oggetto hpc): è lo spazio bidimensionale (piano XY) da tenere in considerazione per ricavare il vano
        di una porta diagonale
    """
    import csv
    with open (fileName, 'rb') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            points = []
            cell = []
            i = 1
            for row in spamreader:
                point1 = [float(row[0])/scale,float(row[1])/scale]
                point2 = [float(row[2])/scale,float(row[3])/scale]
                points.append(point1)
                cell.append(i)
                points.append(point2)
                cell.append(i+1)
                i += 2
    diagonalDoorSpace = MKPOL([points,[cell],1])
    return diagonalDoorSpace

Utilizzo le funzioni definite per:
- Realizzare mura 3D a partire dal file "base/myPlan.lines"
- Ricavare vani finestra a partire dai file "windows/windowsX.lines" (finestre parallele all'asse X) e "windows/windowsY.lines" (finestre parallele all'asse Y)
- Ricavare vani porta a partire dai file "doors/doorsX.lines" (porte parallele all'asse X), "doors/doorsY.lines" (porte parallele all'asse Y) e "diagonalDoors/doorD"+n+".lines" (porte diagonali)

###### n.b. Introdurre una "differenza" di spazi obliqui ha introdotto ritardi nell'elaborazione dell'immagine da parte del kernel

In [157]:
 
house = OFFSET([.1, .1, 3])(lines2structure('base/myPlan.lines'))
#VIEW(house)

windowsX = OFFSET([0, .4, 1.5])(lines2structure('windows/windowsX.lines'))
windowsX = COLOR(RED)(STRUCT([T([2,3])([-0.2,1]),windowsX]))
#VIEW(windowsX)

windowsY = OFFSET([.4, 0, 1.5])(lines2structure('windows/windowsY.lines'))
windowsY = COLOR(RED)(STRUCT([T([1,3])([-0.2,1]),windowsY]))
#VIEW(windowsY)

doorsX = OFFSET([0, .4, 2])(lines2structure('doors/doorsX.lines'))
doorsX= COLOR(RED)(STRUCT([T([2])([-0.2]),doorsX]))
#VIEW(doorsX)

doorsY = OFFSET([.4, 0, 2])(lines2structure('doors/doorsY.lines'))
doorsY = COLOR(RED)(STRUCT([T([1])([-0.2]),doorsY]))
#VIEW(doorsY)

doorD1 =  COLOR(RED)(PROD([diagonalDoor('diagonalDoors/doorD1.lines'),QUOTE([2])]))
doorD2 =  COLOR(RED)(PROD([diagonalDoor('diagonalDoors/doorD2.lines'),QUOTE([2])]))
doorD3 =  COLOR(RED)(PROD([diagonalDoor('diagonalDoors/doorD3.lines'),QUOTE([2])]))
doorD4 =  COLOR(RED)(PROD([diagonalDoor('diagonalDoors/doorD4.lines'),QUOTE([2])]))
doorD5 =  COLOR(RED)(PROD([diagonalDoor('diagonalDoors/doorD5.lines'),QUOTE([2])]))
doorD6 =  COLOR(RED)(PROD([diagonalDoor('diagonalDoors/doorD6.lines'),QUOTE([2])]))
doorsD = STRUCT([doorD1,doorD2,doorD3,doorD4,doorD5,doorD6])

house = STRUCT([house,windowsX,windowsY,doorsX,doorsY,doorD1,doorD2,doorD3,doorD4,doorD5,doorD6])
#VIEW(house)

house = DIFFERENCE([house,windowsX,windowsY,doorsX,doorsY])
#VIEW(house)

house = DIFFERENCE([house,doorsD])
VIEW(house)

house = COLOR([255.0/255,229.0/255,204.0/255])(house)


Ridefinisco funzioni di supporto sviluppate nei workshop precedenti per realizzare porte/finestre (dal workshop_07) e ruotarli (dal workshop_05)

In [158]:
def rotateXY(obj, degrees) :
    obj = STRUCT([R([1,2])(degrees*6.28/360),obj])
    return obj

def sumAllElements(myList):

    s = 0
    for val in myList:
        s += val
    return s

def adaptListToDimension(myList,dim):

    summatory = sumAllElements(myList)
    diff = summatory - dim
    diff = diff / len(myList)
    correctList = []
    if summatory is not dim:
        for val in myList : correctList.append(val*(dim/summatory))
        return correctList
    return myList


def doorExtended (Xlist,Ylist,Zlist,occupancy,colorList):

    def door0(dx,dy,dz):

        
        def door1(cod):

            
            X = adaptListToDimension(Xlist,dx)
            Y = adaptListToDimension(Ylist,dy)
            Z = adaptListToDimension(Zlist,dz)
        
            occ = occupancy

            structure = CUBOID([0,0,0])



            sumX = 0
            k = 0

            for plan in occ:

                page = CUBOID([0,0,0])

                sumZ = 0
                i  = 0

                for row in plan :
                    tmpY = []
                    j=0

                    allNegative = 1

                    for val in row:

                        if val == cod: 
                            tmpY.append(Y[j])
                            allNegative = 0

                        else: tmpY.append(-Y[j])

                        j+=1

                    if not allNegative:

                        qy = QUOTE(tmpY)
                        qz = QUOTE([Z[i]])
                        pxy = PROD([QUOTE([X[k]]),qy])
                        pyz = PROD([pxy,qz])

                        page = STRUCT([page,T([3])([sumZ]),pyz])

                    sumZ += Z[i]
                    i += 1

                structure = STRUCT([structure,T([1])([sumX]),page])

                sumX += X[k]
                k += 1

            return structure
        
        totalStructure = CUBOID([0,0,0])
        
        c = 0
        for val in colorList :
            partialStructure = COLOR(colorList[c])(door1(c+1))
            totalStructure = STRUCT([totalStructure,partialStructure])
            c += 1
        
        return totalStructure

    return door0

woodColor = [94.0/255,41.0/255,0.0/255]
glassColor = [195.0/255,248.0/255,255.0/255]
steelColor = [192.0/255,192.0/255,192.0/255]
darkSteelColor = [96.0/255,96.0/255,96.0/255]
blackWoodColor = [53.0/255,53.0/255,53.0/255]
ornamentalRedColor = [128.0/255,18.0/255,18.0/255]
ornamentalBrownColor = [119.0/255,84.0/255,52.0/255]

Riproduco una finestra (workshop_07)

In [159]:

windowX = [0.08,0.06,0.04,0.06,0.04,0.06,0.08,0.1]
windowY = [0.02,0.1,0.08,0.15,0.03,0.8,0.03,0.15,0.08,0.1,0.02]
windowZ = [0.12,0.15,0.15,0.03,2.2,0.03,0.15,0.12,0.15]

page1 = [[0,3,3,3,3,3,3,3,3,3,0],[0,3,0,0,0,0,0,0,0,3,0],[0,3,0,0,0,0,0,0,0,3,0],
         [0,3,0,0,0,0,0,0,0,3,0],[0,3,0,0,0,0,0,0,0,3,0],[0,3,0,0,0,0,0,0,0,3,0],
         [0,3,0,0,0,0,0,0,0,3,0],[0,3,0,0,0,0,0,0,0,3,0],[0,3,3,3,3,3,3,3,3,3,0]]

page2 = [[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,0,0,0,0,0,3,3,0],
         [0,3,3,0,0,0,0,0,3,3,0],[0,3,3,0,0,0,0,0,3,3,0],[0,3,3,0,0,0,0,0,3,3,0],
         [0,3,3,0,0,0,0,0,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0]]

page3 = [[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,0,3,0,3,0,3,3,0],
         [0,3,3,3,3,3,3,3,3,3,0],[0,3,3,0,3,0,3,0,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],
         [0,3,3,0,3,0,3,0,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0]]

page4 = [[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,4,3,4,3,4,3,3,0],
         [0,3,3,3,3,3,3,3,3,3,0],[0,3,3,4,3,4,3,4,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],
         [0,3,3,4,3,4,3,4,3,3,0],[0,3,3,3,3,3,3,3,3,3,0],[0,3,3,3,3,3,3,3,3,3,0]]

page5 = [[1,1,1,1,1,1,1,1,1,1,1],[0,1,2,2,2,2,2,2,2,1,0],[0,1,2,0,2,0,2,0,2,1,0],
         [0,1,2,2,2,2,2,2,2,1,0],[0,1,2,0,2,0,2,0,2,1,0],[0,1,2,2,2,2,2,2,2,1,0],
         [0,1,2,0,2,0,2,0,2,1,0],[0,1,2,2,2,2,2,2,2,1,0],[1,1,1,1,1,1,1,1,1,1,1]]

page6 = [[1,1,1,1,1,1,1,1,1,1,1],[0,1,2,2,2,2,2,2,2,1,0],[0,1,2,0,0,0,0,0,2,1,0],
         [0,1,2,0,0,0,0,0,2,1,0],[0,1,2,0,0,0,0,0,2,1,0],[0,1,2,0,0,0,0,0,2,1,0],
         [0,1,2,0,0,0,0,0,2,1,0],[0,1,2,2,2,2,2,2,2,1,0],[1,1,1,1,1,1,1,1,1,1,1]]

page7 = [[1,1,1,1,1,1,1,1,1,1,1],[0,1,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,0,0,1,0],
         [0,1,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,0,0,1,0],
         [0,1,0,0,0,0,0,0,0,1,0],[0,1,0,0,0,0,0,0,0,1,0],[1,1,1,1,1,1,1,1,1,1,1]]

page8 = [[1,1,1,1,1,1,1,1,1,1,1],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0]]

pages = [page1,page2,page3,page4,page5,page6,page7,page8]

colorList = [ornamentalBrownColor,ornamentalRedColor,woodColor,glassColor]



Aggiungo al modello le finestre parallele all'asse Y

In [160]:

x1 = 88.531839
y1 = 665.12382
x2 = 88.693985
y2 = 586.4829
windowDX= 0.3
windowDY= math.fabs(y1 - y2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,180)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 59.994103
y1 = 228.95048
x2 = 60.15625
y2 = 309.21286
windowDX= 0.3
windowDY= math.fabs(y1 - y2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,180)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 742.954
y1 = 299.80838
x2 = 742.7918599999999
y2 = 352.18161
windowDX= 0.3
windowDY= math.fabs(y1 - y2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/5,-windowDY/2,1.0]),window])
window = rotateXY(window,0)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 742.79186
y1 = 474.60201
x2 = 743.1161500000001
y2 = 513.8414
windowDX= 0.3
windowDY= math.fabs(y1 - y2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/5,-windowDY/2,1.0]),window])
window = rotateXY(window,0)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 742.62971
y1 = 545.94635
x2 = 743.1161500000001
y2 = 584.0507100000001
windowDX= 0.3
windowDY= math.fabs(y1 - y2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/5,-windowDY/2,1.0]),window])
window = rotateXY(window,0)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

#VIEW(house)


Aggiungo al modello le finestre parallele all'asse X

In [161]:

x1 = 98.422759
y1 = 879.80543
x2 = 179.00943
y2 = 879.64328
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/5,-windowDY/2,1.0]),window])
window = rotateXY(window,90)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 80.586674
y1 = 208.6822
x2 = 137.17571
y2 = 208.6822
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 209.97936
y1 = 91.936917
x2 = 249.21875
y2 = 92.099063
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 262.0283
y1 = 92.099063
x2 = 302.07842
y2 = 92.099063
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 315.05012
y1 = 91.774771
x2 = 354.2895
y2 = 91.93691700000001
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 366.77476
y1 = 91.936917
x2 = 406.33844
y2 = 91.77477099999999
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 460.65743
y1 = 33.077837
x2 = 500.22110999999995
y2 = 33.239984
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 600.58962
y1 = 33.40213
x2 = 640.1533
y2 = 33.40213
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/2,-windowDY/2,1.0]),window])
window = rotateXY(window,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 344.1934
y1 = 822.76495
x2 = 382.94668
y2 = 822.99426
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/5,-windowDY/2,1.0]),window])
window = rotateXY(window,90)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

x1 = 384.78116
y1 = 822.76495
x2 = 424.22237
y2 = 822.99426
windowDX= 0.3
windowDY= math.fabs(x1 - x2)/scale
windowDZ=1.5
window = doorExtended(windowX,windowY,windowZ,pages,colorList)(windowDX,windowDY,windowDZ)
window = STRUCT([T([1,2,3])([-windowDX/5,-windowDY/2,1.0]),window])
window = rotateXY(window,90)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),window])

#VIEW(house)

Riproduco una porta (workshop_07)

In [162]:

doorX = [0.04,0.03,0.03,0.03,0.08,0.03,0.03,0.03,0.04]
doorY = [0.1,0.02,0.005,0.15,0.25,0.5,0.25,0.15,0.005,0.02,0.1]
doorZ = [0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.005,0.02,0.1]

page1 = [[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,1,1,1,1,1,1,1,1,1,1]]

page2 = [[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page3 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page4 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,0,0,1,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,0,0,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page5 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,2,2,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,2,2,1,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,2,2,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page6 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,0,0,1,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,1,0,0,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page7 = [[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,1,1,1,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,1,1,1,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page8 = [[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],[1,1,0,1,0,0,0,1,0,1,1],
         [1,1,0,1,0,0,0,1,0,1,1],[1,1,0,0,0,0,0,0,0,1,1],[1,1,1,1,1,1,1,1,1,1,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


page9 = [[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,0,0,0,0,0,0,0,0,0,1],[1,1,0,0,0,0,0,0,0,1,1],[1,0,0,0,0,0,0,0,0,0,1],
         [1,1,1,1,1,1,1,1,1,1,1]]


pages = [page1,page2,page3,page4,page5,page6,page7,page8,page9]

colorList = [woodColor,glassColor]

Aggiungo al modello le porte parallele all'asse Y

In [163]:

x1 = 420.9316
y1 = 223.43751
x2 = 420.9316
y2= 270.45991
doorDX= 0.1
doorDY= math.fabs(y1 - y2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,180)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 561.99882
y1 = 311.32076
x2 = 561.99882
y2 = 282.13444
doorDX= 0.1
doorDY= math.fabs(y1 - y2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,180)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 550.0
y1 = 381.69222
x2 = 550.0
y2 = 413.47288000000003
doorDX= 0.1
doorDY= math.fabs(y1 - y2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,180)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 451.41509
y1 = 369.6934
x2 = 451.0908
y2 = 409.58137
doorDX= 0.1
doorDY= math.fabs(y1 - y2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,180)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 549.6757
y1 = 440.06486
x2 = 550.32429
y2 = 469.25118000000003
doorDX= 0.1
doorDY= math.fabs(y1 - y2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,180)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 345.69575
y1 = 544.81132
x2 = 346.02004
y2 = 503.95048
doorDX= 0.1
doorDY= math.fabs(y1 - y2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,180)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

#VIEW(house)

Aggiungo al modello le porte parallele all'asse X

In [164]:
x1 = 680.36556
y1 = 232.1934
x2 = 710.52476
y2 = 232.1934
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 459.5224
y1 = 349.58727
x2 = 500.05896
y2 = 349.26297999999997
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 476.7099
y1 = 494.2217
x2 = 505.57193
y2 = 494.2217
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 471.19693
y1 = 559.08019
x2 = 505.24764
y2 = 559.40449
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 418.01297
y1 = 559.40449
x2 = 453.36085
y2 = 559.40449
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 360.28891
y1 = 559.40449
x2 = 389.15094
y2 = 559.08019
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 336.61556
y1 = 641.45048
x2 = 389.15094
y2 = 640.80189
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 471.52122
y1 = 693.98585
x2 = 505.57193
y2 = 693.33727
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 610.64268
y1 = 692.68868
x2 = 669.66391
y2 = 692.68868
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 494.87028
y1 = 1032.5472
x2 = 715.71344
y2 = 1032.5472
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,[woodColor,woodColor])(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 701.44457
y1 = 740.03538
x2 = 737.76532
y2 = 740.03538
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 242.89505
y1 = 764.68161
x2 = 290.24174
y2 = 764.35731
doorDX= 0.1
doorDY= math.fabs(x1 - x2)/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,270)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])


Aggiungo al modello le porte "diagonali"

In [165]:

x1 = 157.93042
y1 = 201.70991
x2 = 187.44104
y2 = 173.17218
doorDX= 0.1
doorDY= math.sqrt(math.pow(x1-x2,2)+math.pow(y1-y2,2))/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,225)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 618.75
y1 = 266.24411
x2 = 642.74764
y2 = 238.35495999999998
doorDX= 0.1
doorDY= math.sqrt(math.pow(x1-x2,2)+math.pow(y1-y2,2))/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,225)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 554.54009
y1 = 497.46463
x2 = 580.15919
y2 = 523.40802
doorDX= 0.1
doorDY= math.sqrt(math.pow(x1-x2,2)+math.pow(y1-y2,2))/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,-45)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 555.18868
y1 = 561.35024
x2 = 579.8349
y2 = 537.02831
doorDX= 0.1
doorDY= math.sqrt(math.pow(x1-x2,2)+math.pow(y1-y2,2))/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,225)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 95.990565
y1 = 483.52005
x2 = 119.01533
y2 = 506.22052
doorDX= 0.1
doorDY= math.sqrt(math.pow(x1-x2,2)+math.pow(y1-y2,2))/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,-45)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

x1 = 175.76651
y1 = 717.98349
x2 = 217.27594000000002
y2 = 758.52005
doorDX= 0.1
doorDY= math.sqrt(math.pow(x1-x2,2)+math.pow(y1-y2,2))/scale
doorDZ=2.0
door = doorExtended(doorX,doorY,doorZ,pages,colorList)(doorDX,doorDY,doorDZ)
door = STRUCT([T([1,2,3])([-doorDX,-doorDY/2,0]),door])
door = rotateXY(door,-45)
house = STRUCT([house,T([1,2])([(x1+x2)/(2*scale),(y1+y2)/(2*scale)]),door])

VIEW(house)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000007D173C0> >

Aggiungo al modello la "veranda grande"

In [166]:
h1 = 2.5
h4 = 3.0
h2 = h1 + ((h4-h1)/(920.15765-764.06174))*(920.15765-879.86298)
h3 = h1 + ((h4-h1)/(920.15765-764.06174))*(920.15765-823.22357)

bigPatioPoints = [[208.44224/scale,920.38696/scale,h1],[208.44224/scale,920.38696/scale,h1-.1],
                  [208.67155/50,879.86298/50,h2],[208.67155/scale,879.86298/scale,h2-.1],
                  [222.2008/scale,879.86298/scale,h2],[222.2008/scale,879.86298/scale,h2-.1],
                  [222.65941999999998/scale,764.06174/scale,h4],[222.65941999999998/scale,764.06174/scale,h4-.1],
                  [310.0263/scale,764.06174/scale,h4],[310.0263/scale,764.06174/scale,h4-.1],
                  [310.25561/scale,823.22357/scale,h3],[310.25561/scale,823.22357/scale,h3-.1],
                  [462.28773/scale,822.9942599999999/scale,h3],[462.28773/scale,822.9942599999999/scale,h3-.1],
                  [462.51704/scale,920.15765/scale,h1],[462.51704/scale,920.15765/scale,h1-.1],
                  [222.65941999999998/scale,920.15765/scale,h1],[222.65941999999998/scale,920.15765/scale,h1-.1],
                  [310.25561/scale,920.15765/scale,h1],[310.25561/scale,920.15765/scale,h1-.1]]
                                     
bigPatio = COLOR(woodColor)(MKPOL([bigPatioPoints,[[1,2,3,4,5,6,17,18],[7,8,9,10,17,18,19,20],[11,12,13,14,15,16,19,20]],1]))
c = COLOR(woodColor)(CUBOID([0.2,0.1,2.5]))
bigPatio = STRUCT([bigPatio,T([1,2])([208.44224/scale,920.15765/scale-0.2]),c,T(1)(2.2),c,T(1)(2.6),c])
house = STRUCT([house,T([1,2])([.03,.03]),bigPatio])


Aggiungo al modello la "veranda piccola"

In [167]:
h1 = 2.5
h2 = 3.0
h3 = 2.7

littlePatioPoints = [[59.831997/scale,208.68247/scale,h3],[59.831997/scale,208.68247/scale,h3-.1],
                    [59.831997/scale,92.260938/scale,h1],[59.831997/scale,92.260938/scale,h1-.1],
                    [193.2783/scale,92.260938/scale,h3],[193.2783/scale,92.260938/scale,h3-.1],
                    [193.2783/scale,166.03774/scale,h2],[193.2783/scale,166.03774/scale,h2-.1],
                    [150.95814/scale,208.68247/scale,h2],[150.95814/scale,208.6824/scale,h2-.1]]

littlePatio = COLOR(woodColor)(MKPOL([littlePatioPoints,[[1,2,3,4,9,10],[3,4,9,10,7,8],[3,4,5,6,7,8]],1]))
c = COLOR(woodColor)(CUBOID([0.2,0.2,2.5]))
littlePatio = STRUCT([littlePatio,T([1,2])([59.831997/scale+0.1,92.260938/scale+0.1]),c])
house = STRUCT([house,littlePatio])


Definisco due funzioni. Una per ottenere una lista di punti a partire da un file ".points" (coordinate 2D di un singolo punto per ogni riga). L'altra per realizzare un sottile piano, a partire dalla lista di punti e dalle celle convesse di cui è composto. A questo parallelepipedo viene applicata una texture (viene specificato come parametro il nome del file)

In [168]:
texDepth = 0.05

def points2list(fileName):
    """
    Funzione che legge coordinate di punti dalle righe di un file e vi realizza una lista
    
    Args:
        fileName(stringa): è il path+nome_del_file da cui la funzione estrapola le coordinate dei punti da inserire nella lista
        
    Returns:
        points(lista di punti): è la lista dei punti letti da file
    """
    import csv
    points = []
    with open (fileName, 'rb') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for row in spamreader:
                x = (float(row[0]))/scale
                y = (float(row[1]))/scale
                points.append([x,y])
    return points
                 

def applyTexture(pointsFileName,textureFileName,cells,n) :
    """
    Funzione crea un sottile piano e vi applica una texture
    
    Args:
        pointsFileName(stringa): è il path+nome_del_file da cui la funzione estrapola le coordinate dei punti da utilizzare
        textureFileName(stringa): è l' immagine che servirà da texture
        cells(lista di celle): sono le celle convesse che compongono la proiezione del parallelepipedo sul piano XY
        n(numero reale): è il numero di quadrati in cui deve essere scomposto il parallelepipedo sulle X
            (il numero dei quadrati sulle y è dipendente dalla grandezza dei quadrati;
             ad ogni quadrato viene applicata la texture)
        
    Returns:
        textured(oggetto hpc) : è il piano opportunatamente ornato di texture
    """
    points = points2list(pointsFileName)
    plan =MKPOL([points,cells,1])
    plan = PROD([plan,QUOTE([texDepth])])

    minx = points[0][0]
    maxx = points[0][0]
    miny = points[0][1]
    maxy = points[0][1]

    for val in points :
        if val[0] < minx : minx = val[0]
        if val[0] > maxx : maxx = val[0]
        if val[1] < miny : miny = val[1]
        if val[1] > maxy : maxy = val[1]

    dim = (maxx - minx)/n

    myList = []

    i = minx
    while i < maxx:
        j = miny
        while j < maxy:
            if j+dim < maxy : myList.append(STRUCT([T([1,2])([i,j]),CUBOID([dim,dim,texDepth])]))
            else :  myList.append(STRUCT([T([1,2])([i,j]),CUBOID([dim,maxy-j,texDepth])]))
            j += dim
        i += dim

    textured = CUBOID([0,0,0])

    i = 0
    for val in myList:
        val = TEXTURE(textureFileName)(INTERSECTION([plan,val]))
        textured = STRUCT([textured,val])
    return textured

parquetCells = [[1,2,3,19],[3,19,14,18],[14,18,17,15,16],[3,4,13,14],[13,4,5,6],[11,12,13,6],[6,9,10,11],[6,7,8,9]]
parquet = applyTexture('texture/perimetro.points','texture/texture.jpg',parquetCells,4.0)

grassCells = [[1,12,13,14],[1,12,18,15],[12,11,18],[11,10,9,18],[18,9,8,7],[18,7,6,17],[17,6,5,16],[5,16,4],[16,4,15],[4,3,2,15],[2,15,1]]
grass =  applyTexture('texture/garden.points','texture/grass.jpg',grassCells,16.0)

house = STRUCT([CUBOID([18,23,0.2]),T(3)(0.2),parquet,grass,T(3)(texDepth),house])
house = STRUCT([house,T(3)(0.2),parquet])


Visualizzo il risultato finale

In [169]:
VIEW(house)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x0000000007A8B2A0> >